### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.


In [2]:
# imports

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [23]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
# openai = OpenAI()
# MODEL = 'gpt-4o-mini'
openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

API key looks good so far


In [24]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [25]:
web = Website("https://www.linkedin.com")
web.links

['#main-content',
 '/legal/user-agreement?trk=linkedin-tc_auth-button_user-agreement',
 '/legal/privacy-policy?trk=linkedin-tc_auth-button_privacy-policy',
 '/legal/cookie-policy?trk=linkedin-tc_auth-button_cookie-policy',
 'ms-windows-store://pdp/?ProductId=9WZDNCRFJ4Q7&mode=mini&cid=guest_desktop_upsell',
 '/?trk=guest_homepage-basic_nav-header-logo',
 'https://www.linkedin.com/top-content?trk=guest_homepage-basic_guest_nav_menu_topContent',
 'https://www.linkedin.com/pub/dir/+/+?trk=guest_homepage-basic_guest_nav_menu_people',
 'https://www.linkedin.com/learning/search?trk=guest_homepage-basic_guest_nav_menu_learning',
 'https://www.linkedin.com/jobs/search?trk=guest_homepage-basic_guest_nav_menu_jobs',
 'https://www.linkedin.com/games?trk=guest_homepage-basic_guest_nav_menu_games',
 'ms-windows-store://pdp/?ProductId=9WZDNCRFJ4Q7&mode=mini&cid=guest_nav_upsell&trk=guest_homepage-basic_guest_nav_menu_windows',
 'https://www.linkedin.com/signup?trk=guest_homepage-basic_nav-header-joi

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.

It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!


In [26]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [27]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [28]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [29]:
print(get_links_user_prompt(web))

Here is the list of links on the website of https://www.linkedin.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
#main-content
/legal/user-agreement?trk=linkedin-tc_auth-button_user-agreement
/legal/privacy-policy?trk=linkedin-tc_auth-button_privacy-policy
/legal/cookie-policy?trk=linkedin-tc_auth-button_cookie-policy
ms-windows-store://pdp/?ProductId=9WZDNCRFJ4Q7&mode=mini&cid=guest_desktop_upsell
/?trk=guest_homepage-basic_nav-header-logo
https://www.linkedin.com/top-content?trk=guest_homepage-basic_guest_nav_menu_topContent
https://www.linkedin.com/pub/dir/+/+?trk=guest_homepage-basic_guest_nav_menu_people
https://www.linkedin.com/learning/search?trk=guest_homepage-basic_guest_nav_menu_learning
https://www.linkedin.com/jobs/search?trk=guest_homepage-basic_guest_nav_menu_jobs
https://www.linkedin.com/g

In [33]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model='llama3.2',
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        # response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [34]:
web = Website("https://www.linkedin.com")
web.links

['#main-content',
 '/legal/user-agreement?trk=linkedin-tc_auth-button_user-agreement',
 '/legal/privacy-policy?trk=linkedin-tc_auth-button_privacy-policy',
 '/legal/cookie-policy?trk=linkedin-tc_auth-button_cookie-policy',
 'ms-windows-store://pdp/?ProductId=9WZDNCRFJ4Q7&mode=mini&cid=guest_desktop_upsell',
 '/?trk=guest_homepage-basic_nav-header-logo',
 'https://www.linkedin.com/top-content?trk=guest_homepage-basic_guest_nav_menu_topContent',
 'https://www.linkedin.com/pub/dir/+/+?trk=guest_homepage-basic_guest_nav_menu_people',
 'https://www.linkedin.com/learning/search?trk=guest_homepage-basic_guest_nav_menu_learning',
 'https://www.linkedin.com/jobs/search?trk=guest_homepage-basic_guest_nav_menu_jobs',
 'https://www.linkedin.com/games?trk=guest_homepage-basic_guest_nav_menu_games',
 'ms-windows-store://pdp/?ProductId=9WZDNCRFJ4Q7&mode=mini&cid=guest_nav_upsell&trk=guest_homepage-basic_guest_nav_menu_windows',
 'https://www.linkedin.com/signup?trk=guest_homepage-basic_nav-header-joi

In [35]:
get_links("https://www.linkedin.com")

{'links': [{'type': 'About page', 'url': 'https://about.linkedin.com/'},
  {'type': 'Company page',
   'url': 'https://www.linkedin.com/company/linkedin/'},
  {'type': 'Sign up page', 'url': 'https://www.linkedin.com/signup'},
  {'type': 'Login page', 'url': 'https://www.linkedin.com/login'},
  {'type': ' Careers/Jobs home page', 'url': 'https://www.linkedin.com/jobs/'},
  {'type': 'Industry top companies hub',
   'url': 'https://business.linkedin.com/hubs/top-companies/'},
  {'type': 'Learning resources home page',
   'url': 'https://learning.linkedin.com/'},
  {'type': 'Recruiting solutions microsite',
   'url': 'https://business.linkedin.com/talent-solutions'},
  {'type': 'Marketing solutions microsite',
   'url': 'https://business.linkedin.com/marketing-solutions'},
  {'type': 'Sales solutions microsite',
   'url': 'https://business.linkedin.com/sales-solutions'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o


In [13]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [15]:
print(get_all_details("https://www.linkedin.com"))

Found links: {'links': [{'type': 'about page', 'url': 'https://about.linkedin.com/?trk=homepage-basic_directory_aboutUrl'}, {'type': 'careers page', 'url': 'https://www.linkedin.com/company/linkedin/jobs?trk=homepage-basic_directory_careersUrl'}, {'type': 'company page', 'url': 'https://www.linkedin.com/directory/companies?trk=homepage-basic_directory_companyDirectoryUrl'}, {'type': 'blog page', 'url': 'https://blog.linkedin.com/?trk=homepage-basic_directory_blogMicrositeUrl'}, {'type': 'press page', 'url': 'https://press.linkedin.com/?trk=homepage-basic_directory_pressMicrositeUrl'}]}
Landing page:
Webpage Title:
LinkedIn: Log In or Sign Up
Webpage Contents:
Skip to main content
Agree & Join LinkedIn
By clicking Continue to join or sign in, you agree to LinkedIn’s
User Agreement
,
Privacy Policy
, and
Cookie Policy
.
LinkedIn
LinkedIn is better on the app
Don’t have the app? Get it in the Microsoft Store.
Open the app
LinkedIn
Top Content
People
Learning
Jobs
Games
Get the app
Join no

In [16]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."


In [17]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [18]:
get_brochure_user_prompt("Linkedin", "https://www.linkedin.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://about.linkedin.com/?trk=homepage-basic_directory_aboutUrl'}, {'type': 'careers page', 'url': 'https://www.linkedin.com/company/linkedin/jobs?trk=homepage-basic_directory_careersUrl'}, {'type': 'jobs page', 'url': 'https://www.linkedin.com/jobs?trk=homepage-basic_directory_jobsHomeUrl'}, {'type': 'press page', 'url': 'https://press.linkedin.com/?trk=homepage-basic_directory_pressMicrositeUrl'}, {'type': 'blog page', 'url': 'https://blog.linkedin.com/?trk=homepage-basic_directory_blogMicrositeUrl'}, {'type': 'developer page', 'url': 'https://developer.linkedin.com/?trk=homepage-basic_directory_developerMicrositeUrl'}, {'type': 'learning page', 'url': 'https://www.linkedin.com/learning/?trk=homepage-basic_directory_learningHomeUrl'}, {'type': 'products page', 'url': 'https://www.linkedin.com/products?trk=homepage-basic_directory_productsHomeUrl'}]}


'You are looking at a company called: Linkedin\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nLinkedIn: Log In or Sign Up\nWebpage Contents:\nSkip to main content\nAgree & Join LinkedIn\nBy clicking Continue to join or sign in, you agree to LinkedIn’s\nUser Agreement\n,\nPrivacy Policy\n, and\nCookie Policy\n.\nLinkedIn\nLinkedIn is better on the app\nDon’t have the app? Get it in the Microsoft Store.\nOpen the app\nLinkedIn\nTop Content\nPeople\nLearning\nJobs\nGames\nGet the app\nJoin now\nSign in\nWith you every step of your career\nSign in with email\nBy clicking Continue to join or sign in, you agree to LinkedIn’s\nUser Agreement\n,\nPrivacy Policy\n, and\nCookie Policy\n.\nNew to LinkedIn?\nJoin now\nExplore top LinkedIn content\nDiscover relevant posts and expert insights — curated by topic and in one place.\nBusiness Strategy\nCareer\nCommunication\nF

In [19]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [20]:
create_brochure("Linkedin", "https://www.linkedin.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://about.linkedin.com/?trk=homepage-basic_directory_aboutUrl'}, {'type': 'careers page', 'url': 'https://www.linkedin.com/company/linkedin/jobs?trk=homepage-basic_directory_careersUrl'}, {'type': 'jobs page', 'url': 'https://www.linkedin.com/jobs?trk=homepage-basic_directory_jobsHomeUrl'}, {'type': 'press page', 'url': 'https://press.linkedin.com/?trk=homepage-basic_directory_pressMicrositeUrl'}, {'type': 'blog page', 'url': 'https://blog.linkedin.com/?trk=homepage-basic_directory_blogMicrositeUrl'}, {'type': 'help page', 'url': 'https://www.linkedin.com/help/linkedin?lang=en&trk=homepage-basic_directory_helpCenterUrl'}, {'type': 'developer page', 'url': 'https://developer.linkedin.com/?trk=homepage-basic_directory_developerMicrositeUrl'}, {'type': 'learning page', 'url': 'https://www.linkedin.com/learning/?trk=homepage-basic_directory_learningHomeUrl'}, {'type': 'products page', 'url': 'https://www.linkedin.com/products?trk=h

# Welcome to LinkedIn: Where Everyone's a Professional... Except Your Cat!

--- 

**What is LinkedIn?**  
Imagine a vast, digital cocktail party where everyone is wearing their finest suit, sleeves rolled up, and all are ready to network—not just on a personal level, but on a wildly professional one! Welcome to LinkedIn! It's the place where business networking meets socializing, minus the awkward small talk about the weather!

---

## A Glimpse Into Our World:

### **Join the Professional Parade!**  
What do a software developer, a marketing guru, and a project manager have in common? They're all just a click away in their own little corner of the LinkedIn universe! Sign up, and perhaps you’ll even land that coveted engineering job or become the next HR wizard!

### **Secret Weapons for Job Seekers:**
- **Open To Work Feature:** Signal recruiters! It's like raising your hand in class but with less chance of being called on. 
- **Endless Job Listings:** From healthcare to arts and design, if you’re looking for a gig, we’ve got your back! (Disclaimer: There are no jobs available for professional couch potatoes…yet.)

---

## Company Culture: “Work Hard, Play Hard… Personally!”  

- **Learning Never Stops:** We’ve got over 35,000 courses, because who doesn’t want to add “Professional Underwater Basket Weaver” to their resume? 🌊
- **Fun & Games:** Yes, we play games too! Take a break with a good ol' session of Pinpoint or Crossclimb. Because no professional network is complete without a little fun! Who said networking can't be a game?

---

## Who Are Our Customers?  
Pretty much anyone trying to navigate their professional life! From your distant cousin who claims to be an entrepreneur to the Uber driver who secretly writes novels, everyone can benefit from LinkedIn. If you’ve got a career goal, we've got the path to get there!

---

### **Careers with Us:**  
Looking for a fabulous career? Well, look no further! LinkedIn is a haven for ambitious folks (and those who just pretend to be ambitious). Explore opportunities in:

- Engineering
- Marketing
- Sales
- And even something as thrilling as… Administrative Assistance! (Exciting, right?!)

Don't you dare underestimate the power of an admin—the real MVPs of the office!

---

## Connect, Learn, and Laugh with LinkedIn!  

Join us today and become part of the world’s largest professional network—because what's more fun than scrolling through job postings while clutching a cup of coffee in a suit you might only wear once a year? 🔍☕

**Let’s network like nobody's watching** (except for your boss...)!  

---  

**Ready for your professional glow-up?**  
**[Sign Up Now!](#)**

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation


In [21]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [22]:
stream_brochure("Linkedin", "https://www.linkedin.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://about.linkedin.com/?trk=homepage-basic_directory_aboutUrl'}, {'type': 'careers page', 'url': 'https://www.linkedin.com/company/linkedin/jobs?trk=homepage-basic_directory_careersUrl'}, {'type': 'press page', 'url': 'https://press.linkedin.com/?trk=homepage-basic_directory_pressMicrositeUrl'}, {'type': 'blog page', 'url': 'https://blog.linkedin.com/?trk=homepage-basic_directory_blogMicrositeUrl'}, {'type': 'learning page', 'url': 'https://www.linkedin.com/learning/?trk=homepage-basic_directory_learningHomeUrl'}, {'type': 'help page', 'url': 'https://www.linkedin.com/help/linkedin?lang=en&trk=homepage-basic_directory_helpCenterUrl'}]}


# Welcome to the LinkedIn Land of Opportunities! 🚀

### A Brochure for the Ambitious: 
**Caution: It may cause excessive career growth and networking!**

---

**️What is LinkedIn? 🤔**

Imagine a magical place where professionals gather to connect, learn, and land their dream jobs! Welcome to LinkedIn, the social networking site that’s all about getting you hired, connected, and sometimes, just a little bit competitive over who has the most impressive job title—spoiler alert: probably Jennifer from Accounting. 

---

### Our Incredible Culture 🌟

We take our mission seriously: to connect the world’s professionals to make them more productive and successful (while occasionally cracking dad jokes in the break room). Our culture thrives on learning—from business analysis to Cybersecurity (cyber like a ninja, not cyber like a creepy stranger).

> "At LinkedIn, we work hard, but we also play hard! We have games that keep our minds sharp, and no, “checking your LinkedIn notifications” doesn’t count as cardio." — Anonymous Employee (probably).

### Who Are Our Customers? 🕵️‍♂️

- **Job Seekers:** Looking for the right role? We’ve got more job listings than you can shake a resume at! 
- **Recruiters:** Want to find top talent? Dive into our ocean of millions of candidates—just don’t forget your snorkel!
- **Businesses:** Post a job, find the best software tools, and watch your competitors feel the heat. 

---

### Careers at LinkedIn: Join Us! 🌈

Are you an overachiever looking to take over the world, one connection at a time? We hire for roles ranging from **Engineering** to **Human Resources** and even **Cybersecurity Ninja (yes, that’s a real job title).** 

**Why Work With Us?**
- You'll experience networking opportunities galore (hello, free coffee and impromptu pinball tournaments)!
- Monthly professional development courses than you can handle (660+ to choose from—bring your learning cap!)
- Competitive salaries that may allow you to finally afford that oat milk latte. ☕️

---

### Join the Fun! 🎉

So whether you're looking to find a job, network with your coworkers, or learn new skills, join us on LinkedIn—your professional success story starts here! Who knows? Maybe one day your link will be the one inspiring others, like “Jane, the Accounting Extraordinaire” or “Bob, the Data Science Wizard.” 

**P.S.** Don’t forget to download our app! (Because who doesn’t want job opportunities on-the-go?)

---

## Ready to Connect? 
**Skip to the main opportunity now! Click [Here](#) to join the LinkedIn family!** 

*LinkedIn – where conversations today could lead to future opportunities and trendy workplace memes!*